## keras를 이용해서, sequence classification 해보기

- 여기서는 imdb에 쌓여 있는 movie review 데이터를 대상으로 sentiment analysis를 수행합니다.
    - 단어를 bow, tfidf 를 사용해서 벡터화하거나, n-gram을 활용해서, 문맥을 파악하는 식으로 벡터화하거나, 아무튼 그렇게 해서 classifier를 만들고 적용하는 방식은 이전에 많이 했습니다.
- 하지만 여기서는, LSTM을 이용합니다. 뭐, n-gram의 경우처럼 일종의 앞 뒤 문맥을 잘 이용해서 학습한다, 정도로 이해하면 되겠네요.
- 전체 뉴럴넷의 아키텍쳐는 다음과 같습니다.
    - embedding => convolution => maxpooling => LSTM => Dense
    - 중간중간에 dropout은 알아서 잘 넣습니다.

데이터를 읽고 padding을 넣어줌
- 개별 무비리뷰에는 캐릭터나 스트링이 들어가있는 것이 아니라, word vocabulary의 index가 채워져 있음.

In [1]:
import numpy as np 
from keras.datasets import imdb

from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Conv1D, MaxPooling1D
from keras.layers.embeddings import Embedding

from keras.preprocessing import sequence

Using TensorFlow backend.


In [2]:
np.random.seed(7)

In [5]:
"""
개별 movie review에 있는, 모든 단어를 고려하는 것은 무의미하기 때문에 
top_words, 즉 상위 5000개의 단어에 대해서만, 추려냄. 나머지는 필터링
그리고 단어는 index로 표시됨.
"""

max_review_length, top_words = 100, 500
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words = top_words)

ValueError: Object arrays cannot be loaded when allow_pickle=False

In [6]:
import numpy as np
# save np.load
np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

# call load_data with allow_pickle implicitly set to true
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words = top_words)

# restore np.load for future normal usage
np.load = np_load_old

In [7]:
X_train = sequence.pad_sequences(X_train, maxlen = max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen = max_review_length)
print("Read data complete")

Read data complete


뉴럴넷 아키텍쳐를 만들고, 예측결과를 확인.

layer의 구성은 
- Embedding: 단어를 벡터화하고, 이 결과 값을 LSTM에 집어넣어줌. 
    - input_dim에 top_words를 넣어주는데, 아마도 내부에서 자동으로 one-hot vector를 만들어주는 것 같음
    - 현재는 one-hot vector가 아니라, 0, 1, 등 word vocab의 index가 넘어감. 
- Conv1D: 구조적인 특성을 파악하기 위해 여러 filter로 찍어줌.
- MaxPooling1D: convolution으로 찍어낸 정보를 좀 더 특징화함. 
- LSTM: sequential한 정보를 활용
- Dense: classification이므로 output layer을 1칸짜리로 넣어줌. 


In [8]:
embedding_vector_length = 32
model = Sequential([
    Embedding(input_dim=top_words, # 500
              output_dim=embedding_vector_length, # 32
              input_length=max_review_length), 
    Conv1D(filters=32, kernel_size=5, padding='same', activation='relu'), 
    MaxPooling1D(pool_size=2),
    LSTM(50), # 원래는 100, 
    Dropout(0.2), 
    Dense(25, activation='sigmoid'), 
    Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5, batch_size=64)
print("training complete")
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: {:.2f}".format(scores[1]*100))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 32)           16000     
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 100, 32)           5152      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 50, 32)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 50)                16600     
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)      